In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[7],5
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,2e-20
molecule,{'o3': 'atmpro'},o3
ng_adju,-,"[0, 0]"
ng_refs,-,"[2, 5]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.028373                    1 -0.016443
0.000750        2 -0.029882                    2 -0.016907
0.001052        3 -0.029822                    3 -0.015831
0.001476        4 -0.025147                    4 -0.011381
0.002070        5 -0.015298                    5 -0.003141
0.002904        6  0.000418                    6  0.009399
0.004074        7  0.022764                    7  0.026788
0.005714        8  0.052554                    8  0.049757
0.008015        9  0.090594                    9  0.078936
0.011243       10  0.137566                   10  0.115055
0.015771       11  0.194546                   11  0.159089
0.022122       12  0.262192                   12  0.211737
0.031031       13  0.340991                   13  0.273656
0.043528       14  0.460107                   14  0.369196
0.061057       15  0.637544                   15  0.515307
0.085645       16  0.842350                   16  0.690556
0.120136       17  1.059467                   17  0.886269
0.168516       18  1.281938                   18  1.100513
0.236378       19  1.502461                   19  1.328964
0.331549       20  1.717484                   20  1.566712
0.465100       21  1.919230                   21  1.794745
0.652400       22  1.966398                   22  1.897899
0.915100       23  1.657827                   23  1.670331
1.283650       24  1.248061                   24  1.280545
1.800600       25  0.925249                   25  0.935102
2.525700       26  0.647655                   26  0.650685
3.542800       27  0.425198                   27  0.435425
4.969550       28  0.284234                   28  0.299175
6.970850       29  0.210428                   29  0.231515
9.778100       30  0.157681                   30  0.178765
13.715850      31  0.112014                   31  0.128665
19.239350      32  0.065161                   32  0.076139
26.987250      33  0.036001                   33  0.042043
37.855300      34  0.022802                   34  0.024238
53.100050      35 -0.001029                   35 -0.002558
73.887500      36 -0.031344                   36 -0.036578
97.662500      37 -0.057120                   37 -0.067921
121.437500     38 -0.075912                   38 -0.091513
145.212500     39 -0.090697                   39 -0.109152
168.987500     40 -0.103255                   40 -0.122207
192.762500     41 -0.103060                   41 -0.117944
216.537500     42 -0.093096                   42 -0.101898
240.312500     43 -0.082666                   43 -0.086142
264.087500     44 -0.069603                   44 -0.069009
287.862500     45 -0.055837                   45 -0.052881
311.637500     46 -0.040852                   46 -0.037382
335.412500     47 -0.028025                   47 -0.025094
359.187500     48 -0.020243                   48 -0.017796
382.962500     49 -0.014679                   49 -0.012632
406.737500     50 -0.010093                   50 -0.008453
430.512500     51 -0.006359                   51 -0.005128
454.287500     52 -0.003683                   52 -0.002777
478.062500     53 -0.001731                   53 -0.001044
501.837500     54 -0.000020                   54  0.000472
525.612500     55  0.001510                   55  0.001822
549.387500     56  0.002876                   56  0.003026
573.162500     57  0.004074                   57  0.004078
596.937500     58  0.005114                   58  0.004990
620.712500     59  0.006013                   59  0.005774
644.487500     60  0.006765                   60  0.006433
668.262500     61  0.007369                   61  0.006957
692.037500     62  0.007866                   62  0.007388
715.812500     63  0.008279                   63  0.007746
739.587500     64  0.008593                   64  0.008016
763.362500     65  0.008821                   65  0.008210
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -10.995326  0.000000 -10.995326 -10.705761 -5.550920e-08   
0.000624    2     -10.995330  0.000002 -10.995328 -10.705764  1.526846e-06   
0.000876    3     -10.995332  0.000003 -10.995329 -10.705765  2.299590e-06   
0.001229    4     -10.995336  0.000005 -10.995330 -10.705767  3.821399e-06   
0.001723    5     -10.995341  0.000010 -10.995332 -10.705771  6.688740e-06   
0.002417    6     -10.995350  0.000017 -10.995333 -10.705776  1.191616e-05   
0.003391    7     -10.995363  0.000030 -10.995333 -10.705785  2.122290e-05   
0.004757    8     -10.995382  0.000052 -10.995329 -10.705797  3.747303e-05   
0.006672    9     -10.995408  0.000091 -10.995317 -10.705813  6.543221e-05   
0.009359    10    -10.995445  0.000157 -10.995288 -10.705836  1.129737e-04   
0.013128    11    -10.995494  0.000266 -10.995227 -10.705864  1.929883e-04   
0.018415    12    -10.995554  0.000449 -10.995105 -10.705898  3.266291e-04   
0.025830    13    -10.995625  0.000750 -10.994875 -10.705934  5.480396e-04   
0.036232    14    -10.995695  0.001241 -10.994455 -10.705961  9.120950e-04   
0.050823    15    -10.995756  0.002096 -10.993659 -10.705963  1.552511e-03   
0.071291    16    -10.995783  0.003670 -10.992114 -10.705906  2.745303e-03   
0.100000    17    -10.995710  0.006461 -10.989249 -10.705712  4.899781e-03   
0.140271    18    -10.995379  0.011184 -10.984195 -10.705218  8.633965e-03   
0.196760    19    -10.994480  0.018863 -10.975617 -10.704115  1.489631e-02   
0.275997    20    -10.992413  0.030898 -10.961514 -10.701829  2.508488e-02   
0.387100    21    -10.988007  0.049096 -10.938911 -10.697275  4.115268e-02   
0.543100    22    -10.978972  0.075527 -10.903445 -10.688494  6.554101e-02   
0.761700    23    -10.963519  0.110994 -10.852525 -10.673886  1.000835e-01   
1.068500    24    -10.943219  0.150943 -10.792276 -10.654570  1.414780e-01   
1.498800    25    -10.919479  0.190820 -10.728659 -10.631617  1.838034e-01   
2.102400    26    -10.892510  0.230007 -10.662503 -10.605107  2.241609e-01   
2.949000    27    -10.865238  0.267685 -10.597553 -10.577984  2.622993e-01   
4.136600    28    -10.842816  0.305080 -10.537736 -10.555283  3.008604e-01   
5.802500    29    -10.828890  0.347244 -10.481646 -10.540589  3.452110e-01   
8.139200    30    -10.822696  0.399296 -10.423400 -10.533174  4.018862e-01   
11.417000   31    -10.825121  0.462945 -10.362175 -10.534075  4.722047e-01   
16.014700   32    -10.843782  0.542612 -10.301169 -10.551139  5.593505e-01   
22.464000   33    -10.897376  0.645987 -10.251389 -10.603281  6.696661e-01   
31.510500   34    -11.004191  0.791382 -10.212809 -10.709737  8.211812e-01   
44.200100   35    -11.171191  0.992657 -10.178534 -10.879459  1.027342e+00   
62.000000   36    -11.417520  1.236816 -10.180704 -11.136959  1.279447e+00   
85.775000   37    -11.762437  1.493457 -10.268980 -11.511405  1.550869e+00   
109.550000  38    -12.108055  1.678208 -10.429847 -11.902129  1.750286e+00   
133.325000  39    -12.446402  1.802762 -10.643640 -12.297345  1.887747e+00   
157.100000  40    -12.789607  1.890535 -10.899072 -12.703716  1.986679e+00   
180.875000  41    -13.148024  1.958152 -11.189873 -13.125220  2.063972e+00   
204.650000  42    -13.487858  2.007735 -11.480123 -13.514782  2.121332e+00   
228.425000  43    -13.785511  2.043199 -11.742311 -13.843242  2.162785e+00   
252.200000  44    -14.044750  2.069625 -11.975125 -14.117013  2.193929e+00   
275.975000  45    -14.260286  2.089138 -12.171148 -14.334523  2.217066e+00   
299.750000  46    -14.432440  2.104037 -12.328404 -14.501007  2.234604e+00   
323.525000  47    -14.561606  2.118150 -12.443456 -14.621773  2.250081e+00   
347.300000  48    -14.656464  2.134081 -12.522383 -14.708500  2.266127e+00   
371.075000  49    -14.732046  2.152652 -12.5793

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')